URL : https://justkode.kr/deep-learning/pytorch-rnn

#### Parameters
- input_size : Input의 사이즈에 해당하는 수를 입력
- hidden_size : 은닉층의 사이즈에 해당하는 수를 입력
- num_layers : RNN의 은닉층 레이어 갯수를 나타냄
- bias : 바이어스 값 활성화 여부를 선택
- batch_first : True 일때, Output 값의 사이즈는 batch, seq, feature 가 됨, 기본값은 False
- dropout : 드롭아웃 비율을 설정 기본값은 0
- bidirectional : True일때 양방향 RNN이 됨, 기본값은 False

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

#### Word Preprocessing

In [29]:
# sentences = ['i like dog', 'i love coffee', 'i hate milk', 'i like you', 'you like cat', 'you love me', 'you hate coffee', 'you love milk']
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

In [30]:
word_list = list(set(" ".join(sentences).split()))
word_list

['love', 'cat', 'i', 'coffee', 'dog', 'milk', 'like', 'hate', 'you']

In [31]:
word_dict = {w : i for i, w in enumerate(word_list)}
number_dict = {i:w for i, w in enumerate(word_list)}

In [32]:
word_dict

{'love': 0,
 'cat': 1,
 'i': 2,
 'coffee': 3,
 'dog': 4,
 'milk': 5,
 'like': 6,
 'hate': 7,
 'you': 8}

In [33]:
number_dict

{0: 'love',
 1: 'cat',
 2: 'i',
 3: 'coffee',
 4: 'dog',
 5: 'milk',
 6: 'like',
 7: 'hate',
 8: 'you'}

#### Text RNN Parameter

In [34]:
batch_size = len(sentences)
n_step = 2 # 학습하려고 하는 문장의 길이 - 1
n_hidden = 5 # 은닉층 사이즈
n_class = len(word_dict)

In [35]:
def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(np.eye(n_class)[input]) # onthot encoding
        target_batch.append(target)
        
    return input_batch, target_batch

In [36]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype = torch.float32,  requires_grad=True)
target_batch = torch.tensor(target_batch, dtype = torch.int64)

#### Text LSTM

In [37]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size = n_class, hidden_size = n_hidden, dropout = 0.3)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        self.Softmax = nn.Softmax(dim = 1)
        
    def forward(self, hidden_and_cell, X):
        X = X.transpose(0, 1)
        outputs, hidden = self.lstm(X, hidden_and_cell)
        outputs = outputs[-1] # 최종 예측 hidden layer
        model = torch.mm(outputs, self.W) + self.b # 최종 예측, 최종 출력 층
        return model

#### Training

In [38]:
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [39]:
for epoch in range(500):
    hidden = torch.zeros(1, batch_size, n_hidden, requires_grad = True)
    cell = torch.zeros(1, batch_size, n_hidden, requires_grad = True)
    output = model((hidden, cell), input_batch)
    loss = criterion(output, target_batch)
    
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:6f}'.format(loss))
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad = True)
cell = torch.zeros(1, batch_size, n_hidden, requires_grad = True)
predict = model((hidden, cell), input_batch).data.max(1, keepdim = True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 0100 cost = 0.327643
Epoch: 0200 cost = 0.038349
Epoch: 0300 cost = 0.014497
Epoch: 0400 cost = 0.008247
Epoch: 0500 cost = 0.005524
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
